In [36]:
import pandas as pd

In [80]:
#load data
_cpgs  = 'matrix_methylation.tsv'
_genes = 'matrix_rna.tsv'
cpgs = pd.read_csv(_cpgs, sep='\t', index_col=[0])
cpgs = cpgs.dropna(axis=1, how='all')
genes = pd.read_csv(_genes, sep='\t', index_col=[0]) 
genes = genes.dropna(axis=1, how='all')

In [81]:
#load info for coding/non-coding genes
path_gene_types = 'gene_types.txt'
gene_types = pd.read_csv(path_gene_types, index_col=[0], sep='\t')

In [82]:
#get list of coding genes without version info
gene_types['Transcript type'] = gene_types.index
_coding = gene_types.loc[gene_types['Transcript type'] == ("protein_coding")]
__coding = gene_types.loc[gene_types['Transcript type'] == ("protein_coding_CDS_not_defined")]
coding = pd.concat([_coding, __coding])
coding_list = coding['Gene stable ID version'].tolist()
rounded_coding = [x[:15] for x in coding_list]

In [83]:
#convert gene names to names without version info
rounded_genes = [x[:15] for x in list(genes.columns)]
genes = genes.set_axis(rounded_genes, axis=1)

In [84]:
#only use 5000 genes with the highest variance in the gene expression matrix
var = genes.var(axis=0, skipna=True)
sort = var.sort_values(ascending=False)
sort5000 = sort[:5000]
list5000 = list(sort5000.index)
genes = genes.loc[:, genes.columns.intersection(list5000)]

In [90]:
#set genes and cpgs together to the expression matrix
cpgs.reset_index(drop=True, inplace=True)
genes.reset_index(drop=True, inplace=True)
ex_matrix = pd.concat([genes, cpgs],axis=1)

In [91]:
#get transcription factor (genes) names and target (cpgs) names
tf_names = pd.DataFrame(genes.columns)
tf_names.index.name = None
target_names = pd.DataFrame(cpgs.columns)
target_names.index.name = None

In [92]:
#save to .tsv file
ex_matrix.to_csv('ex_matrix.tsv', sep='\t')
tf_names.to_csv('tf_names.tsv', sep='\t', index = False, header = False)
target_names.to_csv('target_names.tsv', sep='\t', index = False, header = False)